# Trending YouTube Categories insights
-------------------------------------------------------------------------------------

## Imports

In [466]:
import numpy as np
import pandas as pd
from datetime import date

In [467]:
# Read the YouTube DataSet and Categories from JSON file
json_path = '/Users/matthew/Desktop/python_project/US_category_id.json'
yt_df = pd.read_csv('/Users/matthew/Desktop/python_project/USvideos.csv')
yt_json = pd.read_json(json_path, orient='columns')
cat_json = pd.read_json((yt_json['items']).to_json() , orient='index')
categories_table = pd.read_json((cat_json['snippet']).to_json() , orient='index')

# group id with value to create dictionary
cat_ids = cat_json['id']
cat_titles = categories_table['title']

# contains category id as the key and category as the value
cat_dict = dict(zip(cat_ids, cat_titles))


## Add columns to data frame (categories, number_of_tags, days_before_trending)

In [468]:

# ------------------------------------ Adding categories ------------------------------------
# storing id column from yt_db
cat_id_list_from_main_db = yt_df['category_id']

# Checks every category id in a row and adds a category as a string
categories_col = [v.lower() for catId in cat_id_list_from_main_db for k, v in cat_dict.items() if catId == k]

# add new column to yt_db 
yt_df['categories'] = categories_col


# ------------------------------------ Adding number_of_tags ------------------------------------

tags_list = list(yt_df['tags'])
tags_list

number_of_tags_col = [len(tag.split('|')) for tag in tags_list]

yt_df['number_of_tags'] = number_of_tags_col


# ------------------------------------ Adding days_before_trending ------------------------------------


# TREND DATES
trend_date = yt_df['trending_date']
publish_date = yt_df['publish_time']
trend_dates = []
publish_dates = []
days_before_trending = []


for i in trend_date:

    trend_date = i.split('.')

    # YEAR
    trend_year = str(20) + trend_date[0]

    # MONTH
    trend_month = trend_date[2]
    if (trend_month[0] == str(0)):
        trend_month = trend_month[1]
    else:
        trend_month = trend_date[2]
        
    # DAY
    trend_day = trend_date[1]
    if (trend_day[0] == str(0)):
        trend_day = trend_day[1]
    else:
        trend_day = trend_date[1]
        
    trend_dates.append(trend_year + "/" + trend_month + "/" + trend_day)
    
    

# PUBLISH DATES

for j in publish_date:
    publish_date = j.split('-')
    
    # YEAR
    publish_year = publish_date[0]
    
    # MONTH
    publish_month = publish_date[1]
    if(publish_month[0] == str(0)):
        publish_month = publish_month[1]
    else:
        publish_month = publish_date[1] 
    
    # DAY
    publish_day = publish_date[2][:2]    

    if(publish_day[0] == str(0)):
        publish_day = publish_day[1]
    else:
        publish_day = publish_date[2][:2]
    
        
    publish_dates.append(publish_year + "/" + publish_month + "/" + publish_day)


videos = zip(publish_dates, trend_dates)
    
# CALCULATE DAYS
for p_date, t_date in videos:
    p_date = p_date.split('/')
    t_date = t_date.split('/')
    
    # PARSE
    p_year = int(p_date[0])
    t_year = int(t_date[0])

    p_month = int(p_date[1])
    t_month = int(t_date[1])
    
    p_day = int(p_date[2])
    t_day = int(t_date[2])
        
    
    upload = date(p_year, p_month, p_day)
    trend = date(t_year, t_month, t_day)
    delta = trend - upload
    days_before_trending.append(delta.days)

yt_df['days_before_trending'] = days_before_trending
yt_df


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,categories,number_of_tags,days_before_trending
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,people & blogs,1,1
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",entertainment,4,1
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,comedy,23,2
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,entertainment,27,1
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,entertainment,14,2
5,gHZ1Qz0KiKM,17.14.11,2 Weeks with iPhone X,iJustine,28,2017-11-13T19:07:23.000Z,"ijustine|""week with iPhone X""|""iphone x""|""appl...",119180,9763,511,1434,https://i.ytimg.com/vi/gHZ1Qz0KiKM/default.jpg,False,False,False,Using the iPhone for the past two weeks -- her...,science & technology,7,1
6,39idVpFF7NQ,17.14.11,Roy Moore & Jeff Sessions Cold Open - SNL,Saturday Night Live,24,2017-11-12T05:37:17.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",2103417,15993,2445,1970,https://i.ytimg.com/vi/39idVpFF7NQ/default.jpg,False,False,False,Embattled Alabama Senate candidate Roy Moore (...,entertainment,42,2
7,nc99ccSXST0,17.14.11,5 Ice Cream Gadgets put to the Test,CrazyRussianHacker,28,2017-11-12T21:50:37.000Z,"5 Ice Cream Gadgets|""Ice Cream""|""Cream Sandwic...",817732,23663,778,3432,https://i.ytimg.com/vi/nc99ccSXST0/default.jpg,False,False,False,Ice Cream Pint Combination Lock - http://amzn....,science & technology,13,2
8,jr9QtXwC9vc,17.14.11,The Greatest Showman | Official Trailer 2 [HD]...,20th Century Fox,1,2017-11-13T14:00:23.000Z,"Trailer|""Hugh Jackman""|""Michelle Williams""|""Za...",826059,3543,119,340,https://i.ytimg.com/vi/jr9QtXwC9vc/default.jpg,False,False,False,"Inspired by the imagination of P.T. Barnum, Th...",film & animation,28,1
9,TUmyygCMMGA,17.14.11,Why the rise of the robots won’t mean the end ...,Vox,25,2017-11-13T13:45:16.000Z,"vox.com|""vox""|""explain""|""shift change""|""future...",256426,12654,1363,2368,https://i.ytimg.com/vi/TUmyygCMMGA/default.jpg,False,False,False,"For now, at least, we have better things to wo...",news & politics,20,1


# Helper Methods

In [696]:
def likeToDislikeRatio(likes, dislikes):
    likesRatio = (likes / (likes + dislikes)) * 100 
    return likesRatio

# likeToDislikeRatio(2334, 3246)


def filterByCategory(category):
    df_filtered = yt_df[yt_df['categories'].str.contains(category)]
    return df_filtered

# filterByCategory('comedy')


def showThumbnails(links):
    #Build HTML as string here
    style = '<style>.column,.row{padding:0 4px}.row{display:flex;flex-wrap:wrap}.column{flex:50%}.column img{margin-top:8px;vertical-align:middle}</style>'

    html = style + '<div class="column"> <div class="row">'

    for link in links:
        html += '<img src=' + link + ' width=100 height=80 style="float:left;width:90px;height:90px;margin:0"/>'
        # Style 1: html += '<img src=' + link + ' width=100 height=80 style="padding:1px;"/>'
        # Style 2: html += '<img src=' + link + ' width=100 height=80 style="float:left;width:80px;height:80px;margin:0"/>'

    html += '</div> </div>'

    # Display HTML
    return HTML(html)


# This method returns a wall of thumbnails (may include repetition [repetition = days trending])
def displayCategoryThumbnailWall(category):
    thumbnail_list = list(filterByCategory(category)['thumbnail_link'])
    return showThumbnails(thumbnail_list[:100])
    
    
# This method returns a wall of thumbnails (no repetitions)
def displayModifiedCategoryThumbnailWall(category):
    modified_thumbnail_list = []
    thumbnail_list = filterByCategory(category)['thumbnail_link']
    
    for thumbnail in thumbnail_list:
        if not thumbnail in modified_thumbnail_list:
            modified_thumbnail_list.append(thumbnail)
    return showThumbnails(modified_thumbnail_list[:100])


# Display Thumbnail Wall by taking in a list of thumbnails (can be used for 
def displayCustomListThumbnailWall(thumbnail_list):
    return showThumbnails(thumbnail_list[:100])
    

In [701]:
displayModifiedCategoryThumbnailWall('gaming')

# Work in progress

### Notes:

### Categories: 

In [691]:
cat_set = {""}

for cat in yt_df['categories']:
    cat_set.add(cat)


cat_list = list(cat_set)[1:]
print(cat_list)

df = pd.DataFrame(cat_list)
df

['comedy', 'sports', 'entertainment', 'pets & animals', 'music', 'shows', 'people & blogs', 'gaming', 'autos & vehicles', 'travel & events', 'nonprofits & activism', 'howto & style', 'news & politics', 'education', 'science & technology', 'film & animation']


,0
0,comedy
1,sports
2,entertainment
3,pets & animals
4,music
5,shows
6,people & blogs
7,gaming
8,autos & vehicles
9,travel & events


In [692]:
filterByCategory('gaming')

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,categories,number_of_tags,days_before_trending
131,lsfzA7sWlOM,17.14.11,L.A. Noire - Nintendo Switch Trailer,Nintendo,20,2017-11-09T19:59:48.000Z,"nintendo|""play""|""play nintendo""|""game""|""gamepl...",154872,7683,164,1734,https://i.ytimg.com/vi/lsfzA7sWlOM/default.jpg,False,False,False,Amid the post-war boom of Hollywood's Golden A...,gaming,32,5
392,lsfzA7sWlOM,17.15.11,L.A. Noire - Nintendo Switch Trailer,Nintendo,20,2017-11-09T19:59:48.000Z,"nintendo|""play""|""play nintendo""|""game""|""gamepl...",161231,7803,169,1479,https://i.ytimg.com/vi/lsfzA7sWlOM/default.jpg,False,False,False,Amid the post-war boom of Hollywood's Golden A...,gaming,32,6
405,EEIP1YpFXrE,17.16.11,Our First Date,iHasCupquake,20,2017-11-15T20:00:00.000Z,"first date|""animation""|""animated""|""short""|""sho...",224426,17784,225,2762,https://i.ytimg.com/vi/EEIP1YpFXrE/default.jpg,False,False,False,Here's an animation short of our First Date! W...,gaming,26,1
619,EEIP1YpFXrE,17.17.11,Our First Date,iHasCupquake,20,2017-11-15T20:00:00.000Z,"first date|""animation""|""animated""|""short""|""sho...",463992,25840,509,3708,https://i.ytimg.com/vi/EEIP1YpFXrE/default.jpg,False,False,False,Here's an animation short of our First Date! W...,gaming,26,2
652,hXcoq5XDwyA,17.17.11,Playing CUPHEAD with MatPat!,Butch Hartman,20,2017-11-15T22:02:34.000Z,"game theory|""film theory""|""cuphead""|""don't dea...",55460,3871,96,428,https://i.ytimg.com/vi/hXcoq5XDwyA/default.jpg,False,False,False,"After my appearance on GTLive, MatPat (Game Th...",gaming,15,2
841,EEIP1YpFXrE,17.18.11,Our First Date,iHasCupquake,20,2017-11-15T20:00:00.000Z,"first date|""animation""|""animated""|""short""|""sho...",686170,32999,654,4597,https://i.ytimg.com/vi/EEIP1YpFXrE/default.jpg,False,False,False,Here's an animation short of our First Date! W...,gaming,26,3
875,hXcoq5XDwyA,17.18.11,Playing CUPHEAD with MatPat!,Butch Hartman,20,2017-11-15T22:02:34.000Z,"game theory|""film theory""|""cuphead""|""don't dea...",63841,4226,114,481,https://i.ytimg.com/vi/hXcoq5XDwyA/default.jpg,False,False,False,"After my appearance on GTLive, MatPat (Game Th...",gaming,15,3
1066,EEIP1YpFXrE,17.19.11,Our First Date,iHasCupquake,20,2017-11-15T20:00:00.000Z,"first date|""animation""|""animated""|""short""|""sho...",925595,39452,766,5416,https://i.ytimg.com/vi/EEIP1YpFXrE/default.jpg,False,False,False,Here's an animation short of our First Date! W...,gaming,26,4
1098,hXcoq5XDwyA,17.19.11,Playing CUPHEAD with MatPat!,Butch Hartman,20,2017-11-15T22:02:34.000Z,"game theory|""film theory""|""cuphead""|""don't dea...",68145,4410,115,541,https://i.ytimg.com/vi/hXcoq5XDwyA/default.jpg,False,False,False,"After my appearance on GTLive, MatPat (Game Th...",gaming,15,4
1292,EEIP1YpFXrE,17.20.11,Our First Date,iHasCupquake,20,2017-11-15T20:00:00.000Z,"first date|""animation""|""animated""|""short""|""sho...",1091211,43213,820,5980,https://i.ytimg.com/vi/EEIP1YpFXrE/default.jpg,False,False,False,Here's an animation short of our First Date! W...,gaming,26,5


In [693]:
displayCategoryThumbnailWall('gaming')

# Repetition = Number of days trending